In [1]:
from selenium import webdriver
import json
import time

class Cookies:
    def __init__(self, driver):
        self.driver = driver

    def save_cookies(self):
        cookies = self.driver.get_cookies()
        json_cookies = json.dumps(cookies)
        # 保存cookies，方便后续加载使用
        with open('cookies.json', 'w', encoding='utf-8') as f:
            f.write(json_cookies)

    def add_cookies(self):
        # 加载前先清除之前的cooikes，防止影响
        self.driver.delete_all_cookies()
        with open('cookies.json', 'r', encoding='utf-8') as f:
            list_cookies = json.loads(f.read())
        for i in list_cookies:
            self.driver.add_cookie(i)

In [2]:
import pymysql
from sqlalchemy import create_engine
'''
save_to_mysql方法将数据导入到mysql数据库里
'''
def save_to_mysql(data,table_name):
    engine = create_engine("mysql+pymysql://{}:{}@{}/{}?charset={}".format('root', 'welcome2019', '127.0.0.1:3306', 'WebDataOfTeachers','utf8mb4'))
    con = engine.connect()#创建连接
    data.to_sql(name=table_name, con=con, if_exists='append', index=False)#将dataframe导入到mysql数据库的“teachers”表中。

In [3]:
'''
to_dataframe方法将数据转为dataframe，方便后续数据操作与整理
'''
import pandas
def to_dataframe(data):
    data=pandas.DataFrame(data)#将list类型的数据转为dataframe
    data=data[data['name']!=""]#去除name为空字符串的记录
    return data

In [20]:
def get_data(jobs):
    info=driver.find_elements_by_class_name("job-primary")
    for item in info:
        job={}
        
        job['name']=item.find_element_by_class_name("job-name").text
        job['address']=item.find_element_by_class_name("job-area-wrapper").text
        job['update']=item.find_element_by_class_name("job-pub-time").text
        job['wage']=item.find_element_by_class_name("red").text
        job['degree']=item.find_element_by_class_name("job-limit").find_element_by_tag_name("p").text
        job['company']=item.find_element_by_class_name("company-text").find_element_by_class_name("name").text
        job['company_info']=item.find_element_by_class_name("company-text").find_element_by_tag_name("p").text
        job['tag']=item.find_element_by_class_name("tags").text
        btn=item.find_element_by_class_name("job-title")
        btn.click()
        # 获取当前全部窗口句柄集合；循环句柄并判断是否与当前打开的页面一致，不一致就关闭并切换到一致的窗口句柄
        handles = driver.window_handles
        driver.switch_to.window(handles[1])
        time.sleep(1)
        job['detail']=driver.find_element_by_class_name("job-sec").find_element_by_class_name("text").text
        driver.close()
        driver.switch_to.window(handles[0])
        time.sleep(1)
        jobs.append(job)

In [33]:
url = 'https://www.zhipin.com/job_detail/?ka=header-job'
option = webdriver.ChromeOptions()
# option.add_argument("--headless")
option.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome('chromedriver.exe',options=option)
driver.maximize_window()
driver.implicitly_wait(30) # 隐性等待，最长等30秒
driver.get(url)
# input()
cookies = Cookies(driver)
# 保存cookies
# cookies.save_cookies()
# 加载cookies
cookies.add_cookies()
driver.refresh()
driver.get(url)


In [40]:
url=[
    "https://www.zhipin.com/c101190400-p100599/?ka=sel-city-101190400"
]

In [41]:
for i in url:
    driver.get(i)
    btn=driver.find_element_by_class_name("page").find_element_by_class_name("next")
    count=0
    while 1:
        jobs=[]
        get_data(jobs)
        data=to_dataframe(jobs)
        save_to_mysql(data,"jobs1")
        btn.click()
        time.sleep(3)
        btn=driver.find_element_by_class_name("page").find_element_by_class_name("next")
        count=count+1
        print(count)
        if count==5:
            break


Exception during reset or similar
Traceback (most recent call last):
  File "D:\anaconda\envs\python36\lib\site-packages\pymysql\connections.py", line 713, in _write_bytes
    self._sock.sendall(data)
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\anaconda\envs\python36\lib\site-packages\sqlalchemy\pool\base.py", line 693, in _finalize_fairy
    fairy._reset(pool)
  File "D:\anaconda\envs\python36\lib\site-packages\sqlalchemy\pool\base.py", line 880, in _reset
    pool._dialect.do_rollback(self)
  File "D:\anaconda\envs\python36\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2302, in do_rollback
    dbapi_connection.rollback()
  File "D:\anaconda\envs\python36\lib\site-packages\pymysql\connections.py", line 429, in rollback
    self._execute_command(COMMAND.COM_QUERY, "ROLLBACK")
  File "D:\anaconda\envs\python36\lib\site-packages\pymysql\connec

1
2
3
4
5


In [31]:
data

,name,address,update,wage,degree,company,company_info,tag,detail
0,数据库开发实习生,深圳·龙华区·龙华,发布于09月29日,120-140元/天,5天/周6个月大专,晨科软件,计算机软件不需要融资100-499人,MySQL 数据仓库 数据开发 数据库开发,1.在项目经理的带领下，基于“晨科全业务管理平台”，开发测试ERP、CRM、OA等管理软件。...
1,数据分析,深圳,发布于04月02日,9-14K·15薪,1年以内本科,乐易,游戏不需要融资100-499人,SQL Python R,工作职责：\n负责对游戏道具流转相关数据的分析工作：\n1、构建流转渠道及相关道具的数据字典...
2,数据分析(实习),深圳·龙岗区·坂田,发布于01月18日,200-250元/天,3天/周3个月本科,猜数科技,互联网未融资0-20人,数据分析,工作内容：分析电商平台广告投放数据以优化投放效果，分析商品类目支撑\n产品开发决策，监测社交...
3,NLP 算法工程师 （校招）,深圳·福田区·梅林,发布于03月21日,15-25K·14薪,在校/应届本科,来觅数据,互联网A轮20-99人,Python 深度学习算法 自然语言处理 机器学习算法,岗位职责\n1. 利用 NLP、机器学习技术进行文本挖掘及建模工作；\n2. 设计策略和算法...
4,计划审核专员,深圳·龙华区·民治,发布于04月02日,6-8K,1年以内大专,深圳市利朗达,电子商务B轮1000-9999人,Excel 函数 SQL python Hadoop,岗位职责\n1.审核各销售端每日备货计划；\n2.接收相关部门数据，按时维护和更新数据；\n...
5,数据分析师,深圳·龙岗区·坂田,发布于08月10日,6-8K,1年以内大专,亚飞电商,电子商务不需要融资1000-9999人,数据分析 EXCEL Tableau SAS SPSS,岗位职责\n1、了解业务，针对亚马逊平台的数据进行分析， 定期输出分析报告，例如：销量与流量...
6,物料员,深圳·龙岗区·坂田,发布于03月27日,5-7K,1年以内高中,莱特宁耐科技有限公司,电子商务不需要融资100-499人,物料,工作内容：|\n不错： 笔笔有帐，帐卡物相符；\n不坏：物物可用，质量确保；\n不乱：井然有...
7,数据分析实习生,深圳·南山区·科技园,发布于01月18日,80-120元/天,3天/周3个月本科,点宽科技,互联网未融资20-99人,数据分析 数据挖掘,工作职责：\n1、数据集的整理和处理；\n2、数据分析相关课程开发；\n3、使用python...
8,大数据工程师,深圳·南山区·海岸城,发布于04月12日,7-12K,在校/应届本科,房极客,互联网B轮100-499人,Python Java Shell Scala Spark,岗位职责：\n1、负责大数据平台的开发、测试和维护工作。\n2、负责大数据相关的业务，如业务...
9,技术专员/ 网络管理,深圳·龙华区·龙华,发布于07月17日,4-5K,在校/应届学历不限,小妮子财务,企业服务不需要融资20-99人,数据库开发 网络推广 数据处理 数据采集,1、负责网络推广管理，APP运营管理，数据处理开发及分配，客户资源开发\n2、负责日常开发工...
